In [ ]:
# !pip install wandb

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import wandb
from tqdm import tqdm

In [ ]:
wandb.init(project="Mnist_transfer_learnning")

acc,▁▃▅▇▇▇█▇▇▇██████
loss,█▄▃▂▂▂▁▁▂▁▁▁▁▁▁▁
acc,1.0
loss,0.00134


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet50(pretrained=True)

In [ ]:
in_features = model.fc.in_features
model.fc = nn.Linear(in_features,10)

In [ ]:
# ct = 0
# for child in model.children():
#   ct +=1
#   if ct < 45:
#      for param in child.parameters():
#          param.requires_grad =True

In [ ]:
model = model.to(device)

In [ ]:
config = wandb.config
config.learning_rate = 0.01
config.batch_size =32
config.epochs = 15

In [ ]:
transform = transforms.Compose([
                                
                                # transforms.RandomRotation(10),
                                transforms.Resize((70,70)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

In [ ]:
# dataset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/MNIST_persian",transform = transform )
# train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

In [ ]:
def train_val_dataset(dataset, test_split=0.30):
    train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=test_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['test'] = Subset(dataset, test_idx)
    return datasets

dataset = torchvision.datasets.ImageFolder('/content/drive/MyDrive/MNIST_persian', transform=transform)
print(len(dataset))
datasets = train_val_dataset(dataset)
print(len(datasets['train']))
print(len(datasets['test']))
train_data_loader = torch.utils.data.DataLoader(datasets['train'], batch_size=config.batch_size, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(datasets['test'], batch_size=config.batch_size, shuffle=False)
print(len(train_data_loader))
print(len(test_data_loader))

1200
840
360
27
12


In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=config.learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
def calc_acc(preds,labels):
  _,preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max==labels.data,dtype = torch.float64)/len(preds)
  return acc

In [ ]:
# train
wandb.watch(model)


for epoch in range(config.epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in tqdm(train_data_loader):

    images, labels = images.to(device), labels.to(device)
    # print(images.shape)
    optimizer.zero_grad()
    # images = images.float()
    preds = model(images)

    loss = loss_function(preds,  labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc +=calc_acc(preds, labels)

  total_loss = train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)
  if epoch % 2 == 0:
     wandb.log({"loss": total_loss})
     wandb.log({"acc": total_acc})

  # total_acc = train_acc/len(train_data_loader)
  print(f"Epoch:{epoch}: Loss:{total_loss}, Acc:{total_acc}")
  
  print("-------------------------------------")

100%|██████████| 27/27 [00:06<00:00,  4.22it/s]


Epoch:0: Loss:3.962498903274536, Acc:0.11226851851851852
-------------------------------------


100%|██████████| 27/27 [00:06<00:00,  4.39it/s]


Epoch:1: Loss:2.286302089691162, Acc:0.17129629629629628
-------------------------------------


100%|██████████| 27/27 [00:06<00:00,  4.42it/s]


...

Epoch:12: Loss:0.17082306742668152, Acc:0.9375
-------------------------------------


100%|██████████| 27/27 [00:06<00:00,  4.34it/s]


Epoch:13: Loss:0.15043075382709503, Acc:0.9502314814814814
-------------------------------------


100%|██████████| 27/27 [00:06<00:00,  4.36it/s]

Epoch:14: Loss:0.2883991599082947, Acc:0.9027777777777777
-------------------------------------


In [ ]:

wandb.watch(model)


for epoch in range(config.epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in tqdm(test_data_loader):

    images, labels = images.to(device), labels.to(device)
    # print(images.shape)
    optimizer.zero_grad()
    # images = images.float()
    preds = model(images)

    loss = loss_function(preds,  labels)
    loss.backward()

    optimizer.step()

    test_loss += loss
    test_acc +=calc_acc(preds, labels)

  total_loss = test_loss/len(test_data_loader)
  total_acc = test_acc/len(test_data_loader)
  if epoch % 2 == 0:
     wandb.log({"loss": total_loss})
     wandb.log({"acc": total_acc})

  # total_acc = train_acc/len(train_data_loader)
  print(f"Epoch:{epoch}: Loss:{total_loss}, Acc:{total_acc}")
  
  print("-------------------------------------")

100%|██████████| 12/12 [00:02<00:00,  4.48it/s]


Epoch:0: Loss:0.2706540822982788, Acc:0.9140625
-------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.48it/s]


Epoch:1: Loss:0.15352053940296173, Acc:0.9635416666666666
-------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.51it/s]


...

Epoch:12: Loss:0.0013841481413692236, Acc:1.0
-------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.51it/s]


Epoch:13: Loss:0.0012381405103951693, Acc:1.0
-------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.47it/s]

Epoch:14: Loss:0.0011198443826287985, Acc:1.0
-------------------------------------


In [ ]:
torch.save(model.state_dict(),"persian_mnist.pth")

In [ ]:
transformation = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize((70,70)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
#inference
import cv2
import numpy as np

model.eval()

img = cv2.imread("/content/5.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# img = cv2.resize(img,(28,28))
tensor = transformation(img).unsqueeze(0).to(device)

preds = model(tensor)

preds = preds.cpu().detach().numpy()
print(preds)
output = np.argmax(preds)
output

[[ 1.0763988  -0.5662527  -0.8025147  -1.8023357  -0.41958416  2.5255759
   0.86044455 -3.028757   -1.6737427   2.156714  ]]


5